In [3]:
# general
import pandas as pd
import geopandas

# SQL 
import sqlite3
from sqlalchemy import create_engine

# utility
from tqdm import tqdm

In [4]:
geo = geopandas.read_file("geo/Municipios_Colombia.geojson")
AGROSAVIA_df = pd.read_csv("suelos_geolocalizado.csv")

# Preparación de los datos

## Tabla `municipios`

In [26]:
municipios = geo.set_index("id")[["ID_DEP","departamento","municipio","LATITUD","LONGITUD"]]

#### Capitalizar nombres de municipios

In [32]:
# manually select the stopwords
words = set(" ".join(municipios[["departamento","municipio"]].values.ravel()).split())
" ".join(sorted(
    [word for word in words if len(word) <4],
    key=len
))

'- y de el fé la oro sur paz del los iró iza gil uré san ana río une'

In [33]:
def capitalize_except(capitalize:pd.Series, exceptions:"iterable"):
    results = capitalize.str.title()
    for word in exceptions:
        # the space prevents the replacement of the first word
        results = results.str.replace(" "+word.title().strip()," "+word.lower().strip())
    return results

# capitalize
stopwords = {"y","la","el","de","del","los"}
municipios.municipio = capitalize_except(municipios.municipio,stopwords)
municipios.departamento = capitalize_except(municipios.departamento,stopwords)

In [37]:
# altitude data
municipios["altitud"] = pd.read_csv(
    "geo/altitud.csv",
    dtype={"id":str,"altitud":float}
).set_index("id").altitud.round(2)

# renames
municipios.index.rename("cod_municipio",inplace=True)
municipios.rename(columns={
    "ID_DEP":"cod_departamento","ID_MUN":"cod_municipio",
    "LATITUD":"latitud","LONGITUD":"longitud"
},inplace=True)

In [38]:
municipios.sample(5)

,cod_departamento,departamento,municipio,latitud,longitud,altitud
cod_municipio,,,,,,
54109,54,Norte de Santander,Bucarasica,8.079802,-72.929199,1653.11
68169,68,Santander,Charta,7.258964,-72.987879,2432.86
97001,97,Vaupés,Mitú,1.015572,-70.417553,209.49
50245,50,Meta,El Calvario,4.353770,-73.714417,2366.61
81591,81,Arauca,Puerto Rondón,6.411762,-70.966790,132.47


## Tablas `zonas` y  `departamentos`

In [39]:
zonas = pd.DataFrame({
    "AMA":"Amazonía",
    "AND":"Andina",
    "ORI":"Orinoquía",
    "PAC":"Pacífico",
    "CAR":"Caribe"
}.items(),columns=["cod_region","region"]).set_index("cod_region")
zonas

,region
cod_region,
AMA,Amazonía
AND,Andina
ORI,Orinoquía
PAC,Pacífico
CAR,Caribe


In [40]:
zonas_dict = {
    "AMA":["18","50","86","91","94","95","97","99"],
    "AND":["05","11","15","17","18","19","20","25","27","41","52","54","63","66","68","73","76","86"],
    "ORI":["50","81","85","99"],
    "PAC":["19","27","52","76"],
    "CAR":["08","13","20","23","44","47","70","88"]
}

In [41]:
departamentos = municipios[["cod_departamento","departamento"]].drop_duplicates().set_index("cod_departamento")
departamentos = pd.concat([
    departamentos,
    pd.DataFrame(index=departamentos.index,columns=zonas_dict.keys())
],axis=1)
for col,deps in zonas_dict.items():
    departamentos.loc[deps,col] = 1
departamentos.fillna(0,inplace=True)

In [42]:
# check that all departments have regions
(departamentos[zonas_dict.keys()].sum(axis=1) == 0).sum()

0

In [43]:
departamentos

,departamento,AMA,AND,ORI,PAC,CAR
cod_departamento,,,,,,
05,Antioquia,0,1,0,0,0
08,Atlántico,0,0,0,0,1
11,"Bogotá, D.C.",0,1,0,0,0
13,Bolívar,0,0,0,0,1
15,Boyacá,0,1,0,0,0
17,Caldas,0,1,0,0,0
18,Caquetá,1,1,0,0,0
19,Cauca,0,1,0,1,0
20,Cesar,0,1,0,0,1


## Tabla `analisis`

In [5]:
analisis = AGROSAVIA_df.drop(["departamento","municipio","id"],axis=1)
analisis.index.rename("id",inplace=True)
analisis.rename(columns={"geo_id":"cod_municipio"},inplace=True)

In [6]:
analisis.sample(5)

,cod_municipio,cultivo,fertilizantes,ph,materia_organica,fosforo,azufre,acidez,aluminio,calcio,...,drenaje_regular,riego_aspersión,riego_cañón,riego_goteo,riego_gravedad,riego_manguera,riego_microaspersión,riego_no_cuenta_con_riego,riego_no_indica,riego_por_inundación
id,,,,,,,,,,,,,,,,,,,,,
6830,52411,caña panelera/azucar,no indica,5.93,2.724978,40.600000,3.750000,NaN,NaN,6.930000,...,1,0,0,0,0,0,0,1,0,0
23935,76823,melón,no indica,7.22,1.352156,50.948337,17.479415,NaN,NaN,6.664320,...,0,0,0,1,0,0,0,0,0,0
45257,18256,pastos,no indica,5.01,2.290000,1.935000,2.940000,3.070000,2.330000,1.300000,...,0,0,0,0,0,0,0,0,1,0
26636,15814,no indica,no indica,6.21,6.584613,169.368570,15.702317,NaN,NaN,17.858990,...,0,0,0,0,0,0,0,0,1,0
18139,76243,plátano,no indica,5.28,7.294025,21.756010,7.830300,0.939587,0.667919,3.487647,...,0,0,0,0,0,0,0,1,0,0


In [46]:
analisis.columns

Index(['cod_municipio', 'cultivo', 'fertilizantes', 'ph', 'materia_organica',
       'fosforo', 'azufre', 'acidez', 'aluminio', 'calcio', 'magnesio',
       'potasio', 'sodio', 'cice', 'ce', 'hierro_olsen', 'cobre', 'manganeso',
       'zinc_olsen', 'boro', 'hierro_doble_acido', 'cobre_doble_acido',
       'manganeso_doble_acido', 'zinc_doble_acido', 'estado_establecido',
       'estado_no_indica', 'estado_por_establecer',
       'tiempo_establecimiento_de_0_a_1_año',
       'tiempo_establecimiento_de_1_a_5_años',
       'tiempo_establecimiento_de_5_a_10_años',
       'tiempo_establecimiento_mas_de_10_años',
       'tiempo_establecimiento_no_aplica', 'tiempo_establecimiento_no_indica',
       'topografia_ligeramente_ondulado', 'topografia_moderadamente_ondulado',
       'topografia_no_indica', 'topografia_ondulado', 'topografia_pendiente',
       'topografia_pendiente_fuerte', 'topografia_pendiente_leve',
       'topografia_pendiente_moderada', 'topografia_plano', 'drenaje_bueno',
    

## Tabla `variables`

In [47]:
variables = pd.DataFrame({
    "acidez":"Acidez",
    "aluminio":"Aluminio",
    "azufre":"Azufre",
    "boro":"Boro",
    "calcio":"Calcio",
    "ce":"Coductividad eléctrica",
    "cice":"Coef. intercambio catiónico",
    "cobre":"Cobre",
    "cobre_doble_acido":"Cobre doble ácido",
    "fosforo":"Fósforo",
    "hierro_doble_acido":"Hierro doble ácido",
    "hierro_olsen":"Hierro Olsen",
    "magnesio":"Magnesio",
    "manganeso":"Manganeso",
    "manganeso_doble_acido":"Manganeso doble ácido",
    "materia_organica":"Materia orgánica",
    "ph":"Ph",
    "potasio":"Potasio",
    "sodio":"Sodio",
    "zinc_olsen":"Zinc Olsen",
}.items(),columns=["codigo","variable"]).set_index("codigo")

variables.sample(5)

,variable
codigo,
calcio,Calcio
ce,Coductividad eléctrica
acidez,Acidez
hierro_doble_acido,Hierro doble ácido
ph,Ph


# Creación de la base de datos

In [48]:
tablas = ["municipios","departamentos","zonas","analisis","variables"]

In [49]:
engine = create_engine("sqlite:///database.db")

for name in tablas:
    tabla = locals()[name]
    con = engine.connect()
    
    tabla.to_sql(name, con, if_exists='replace')
    
    con.close()
    print(f"table `{name}` written")

table `municipios` written
table `departamentos` written
table `zonas` written
table `analisis` written
table `variables` written


In [ ]:
!mv database.db ../dashboard/src/data/database.db